# LFT: Introduction (Rigor Edition)

This notebook introduces **Logic Field Theory (LFT)** with **fully justified derivations** and verifications on the minimal nontrivial case **N=3**. We formalize the logical filtering pipeline (A = L(I)), connect it to **Coxeter system** $W(A_{N-1})$, and verify claims with exact code.

### Roadmap
1. **Logical Filtering** (Identity, Non-Contradiction, Excluded Middle) → DAGs/partials/totals.
2. **Coxeter Derivation**: $S_N \cong W(A_{N-1})$ acts on the sum-zero space $V$ (dim = $N-1$).
3. **Permutohedron**: vertices = total orders, **edges = adjacent transpositions**.
4. **Verification for $N=3$**: pattern counts, $A_2$ Gram matrix, Cayley graph properties, and the permutohedron hexagon.


## 1. Logical Filtering: from raw distinctions to orders

**Definition.** A *pattern* on elements $\{0,\dots,N-1\}$ is a set of ordered pairs $E\subseteq \{0,\dots,N-1\}^2$. The logical operator $L$ enforces:
- **Identity (ID)**: $(i,i)$ for all $i$ (reflexivity).
- **Non-Contradiction (NC)**: never both $(i,j)$ and $(j,i)$ for $i\ne j$.
- **Excluded Middle (EM)** *(operational proxy)*: **acyclicity** (so the non-reflexive edges form a DAG).

Thus a pattern passes $L$ iff its non-reflexive edges define a **partial order**. **Total orders** are DAGs where for each unordered pair $\{i,j\}$ exactly one orientation appears.

In [ ]:
import numpy as np
import networkx as nx
from itertools import combinations

def has_cycle(pattern):
    edges = [(a,b) for (a,b) in pattern if a!=b]
    G = nx.DiGraph()
    G.add_edges_from(edges)
    return not nx.is_directed_acyclic_graph(G)

def violates_non_contradiction(pattern):
    pairs = set((a,b) for (a,b) in pattern if a!=b)
    for (a,b) in pairs:
        if (b,a) in pairs:
            return True
    return False

def apply_L(pattern, elements):
    # Identity
    p = set(pattern) | {(i,i) for i in elements}
    # Non-Contradiction
    if violates_non_contradiction(p):
        return None
    # Excluded Middle (acyclicity)
    if has_cycle(p):
        return None
    return p

def is_total_order(pattern, elements):
    p = set((a,b) for (a,b) in pattern if a!=b)
    # exactly one orientation per unordered pair
    for i,j in combinations(elements,2):
        if ((i,j) in p) == ((j,i) in p):
            return False
    # acyclic
    return nx.is_directed_acyclic_graph(nx.DiGraph(list(p)))

def pattern_to_ordering(pattern, elements):
    if not is_total_order(pattern, elements):
        return None
    G = nx.DiGraph([(a,b) for (a,b) in pattern if a!=b])
    order = list(nx.topological_sort(G))
    return '<'.join(map(str, order))

### Exhaustive classification for $N=3$
We enumerate **all directed-pair subsets** (with reflexives enforced by ID) and classify into cycles, partials, and totals.

In [ ]:
from itertools import product
import matplotlib.pyplot as plt
import os

N=3
elements=tuple(range(N))
dirs=[(a,b) for a in elements for b in elements if a!=b]  # 6 directed pairs

all_patterns=[]
for mask in range(1<<len(dirs)):
    subset=[dirs[k] for k in range(len(dirs)) if (mask>>k)&1]
    pattern=set(subset) | {(i,i) for i in elements}
    all_patterns.append(pattern)

filtered=[p for p in all_patterns if apply_L(p, elements) is not None]
totals=[p for p in filtered if is_total_order(p, elements)]
partials=[p for p in filtered if not is_total_order(p, elements)]
cycles=[p for p in all_patterns if has_cycle(p)]

# Generate comprehensive results
results = {
    'total_patterns': len(all_patterns),
    'consistent': len(filtered), 
    'partials': len(partials),
    'totals': len(totals),
    'cycles': len(cycles)
}

print("N=3 Logical Filtering Results:")
print("=============================")
print(f"Total patterns examined: {results['total_patterns']}")
print(f"Cycles (violate NC): {results['cycles']}")  
print(f"Consistent (pass L-filter): {results['consistent']}")
print(f"├─ Partial orders: {results['partials']}")
print(f"└─ Total orders: {results['totals']}")
print(f"\nFeasibility ratio ρ₃ = {results['totals']}/{results['total_patterns']} = {results['totals']/results['total_patterns']:.3f}")

# Validate against theoretical expectations
assert results['totals'] == 6, f"Expected 6 total orders (3!), got {results['totals']}"
assert results['total_patterns'] == 64, f"Expected 64 patterns (2^6), got {results['total_patterns']}"
print("✓ Theoretical validation passed")

# Extract and display the 6 total orders
print(f"\nThe {results['totals']} total orders (permutations of S₃):")
total_orderings = []
for i, pattern in enumerate(totals):
    ordering = pattern_to_ordering(pattern, elements)
    total_orderings.append(ordering)
    print(f"  {i+1}: {ordering}")

os.makedirs('./outputs', exist_ok=True)

# Enhanced pie chart with values
labels=['Cycles','Partials','Totals']
sizes=[len(cycles),len(partials),len(totals)]
colors=['#ff6b6b', '#4ecdc4', '#45b7d1']

plt.figure(figsize=(8,6))
wedges, texts, autotexts = plt.pie(sizes, labels=labels, autopct='%1.1f%%', 
                                   startangle=90, colors=colors, explode=(0.05, 0.05, 0.1))

# Add count annotations
for i, (wedge, size) in enumerate(zip(wedges, sizes)):
    angle = (wedge.theta2 + wedge.theta1) / 2
    x = 1.1 * np.cos(np.radians(angle))
    y = 1.1 * np.sin(np.radians(angle))
    plt.annotate(f'n={size}', xy=(x, y), ha='center', va='center', fontweight='bold')

plt.title('N=3 Pattern Distribution\n(Total patterns = 64)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('./outputs/N3_pattern_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Pattern distribution chart saved to ./outputs/N3_pattern_distribution.png")
print(f"✓ All S₃ permutations recovered: {set(total_orderings) == {'0<1<2', '0<2<1', '1<0<2', '1<2<0', '2<0<1', '2<1<0'}}")

## 2. Coxeter derivation: $S_N \cong W(A_{N-1})$ on $V=\{x: \sum x_i=0\}$

**Theorem (standard).** With simple roots $\alpha_i=e_i-e_{i+1}$, the reflections
$$s_i(x)=x-\frac{2\langle x,\alpha_i\rangle}{\langle\alpha_i,\alpha_i\rangle}\,\alpha_i$$
swap coordinates $i\leftrightarrow i+1$. The generated group is isomorphic to $S_N$, and $\dim V=N-1$ gives the **rank**.

**Check for $A_2$ (N=3):** the Gram/Cartan matrix has diagonals 2, off-diagonals −1 for adjacent roots (cosine −1/2 = 120°).

In [ ]:
import numpy as np

def roots_A(N):
    # simple roots alpha_i = e_i - e_{i+1} in R^N
    roots=[np.eye(N)[i]-np.eye(N)[i+1] for i in range(N-1)]
    roots=np.stack(roots, axis=0)  # (N-1, N)
    G=np.zeros((N-1,N-1))
    for i in range(N-1):
        for j in range(N-1):
            G[i,j]=2*np.dot(roots[i],roots[j])  # Cartan form: 2*(,)/(||alpha||^2) with ||alpha||^2=2
    return roots,G

roots,G=roots_A(3)
print('Cartan A2:\n',G)
cos=G/2.0
print('Cosines:\n',cos)

## 3. Permutohedron & Cayley graph (edges = adjacent swaps)

**Definition.** For strictly increasing $a\in\mathbb{R}^N$ with $\sum a_i=0$, the **permutohedron** $\Pi_{N-1}(a)=\operatorname{conv}\{\sigma\cdot a\}$ lives in $V$. Its 1-skeleton connects permutations that differ by a **single adjacent transposition** (Coxeter generator).

We verify this for $N=3$ by constructing the **adjacent-generator** Cayley graph on $S_3$ and checking degree/edge counts and connectivity.

In [ ]:
import itertools, networkx as nx

def cayley_adjacent_graph(N):
    perms=list(itertools.permutations(range(N)))
    idx={p:i for i,p in enumerate(perms)}
    G=nx.Graph()
    G.add_nodes_from(range(len(perms)))
    gens=[(i,i+1) for i in range(N-1)]
    for p in perms:
        u=idx[p]
        for (i,j) in gens:
            q=list(p); q[i],q[j]=q[j],q[i]
            v=idx[tuple(q)]
            if u<v:
                G.add_edge(u,v)
    return G, perms

G3, perms3 = cayley_adjacent_graph(3)
print({'nodes':G3.number_of_nodes(),'edges':G3.number_of_edges(),'avg_degree':2*G3.number_of_edges()/G3.number_of_nodes()})
assert G3.number_of_nodes()==6
assert G3.number_of_edges()==6  # N!*(N-1)/2 for N=3 → 6*2/2=6
assert nx.is_connected(G3)
print('Cayley S3 (adjacent) checks passed.')

### Visual: permutohedron $\Pi_2$ (N=3) as a hexagon (PCA on $V$)
We embed the vertex set (permutations of a centered vector) in the sum-zero space and plot a 2D PCA projection to show the hexagon structure.

In [ ]:
import numpy as np, matplotlib.pyplot as plt

def sum_zero_basis(N):
    diffs=np.zeros((N,N-1))
    for i in range(N-1):
        diffs[i,i]=1.0; diffs[i+1,i]=-1.0
    U,S,Vt=np.linalg.svd(diffs, full_matrices=False)
    return U  # N x (N-1)

def permutohedron_coords(N):
    B=sum_zero_basis(N)
    a=np.arange(N, dtype=float)-(N-1)/2.0
    verts=[]
    labels=[]
    for p in itertools.permutations(range(N)):
        verts.append(B.T @ a[list(p)])
        labels.append(''.join(map(str,p)))
    V=np.vstack(verts)  # (N!, N-1)
    X=V - V.mean(axis=0, keepdims=True)
    U,S,Vt=np.linalg.svd(X, full_matrices=False)
    P2=X @ Vt[:2].T
    return V,P2,labels

V,P2,vertex_labels=permutohedron_coords(3)

# Create enhanced hexagon visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot: Basic scatter
ax1.scatter(P2[:,0], P2[:,1], s=100, c='#45b7d1', alpha=0.8, edgecolors='black')
for i, label in enumerate(vertex_labels):
    ax1.annotate(label, (P2[i,0], P2[i,1]), xytext=(5, 5), 
                textcoords='offset points', fontsize=10, fontweight='bold')
ax1.set_title('Permutohedron Π₂ (N=3)\nVertices = S₃ permutations', fontsize=12)
ax1.axis('equal')
ax1.grid(True, alpha=0.3)

# Right plot: With edge connections (Cayley graph structure)
ax2.scatter(P2[:,0], P2[:,1], s=100, c='#45b7d1', alpha=0.8, edgecolors='black')

# Add edges from the Cayley graph
G3, perms3 = cayley_adjacent_graph(3)
perm_to_idx = {p: i for i, p in enumerate(perms3)}

for edge in G3.edges():
    i, j = edge
    x_coords = [P2[i,0], P2[j,0]]
    y_coords = [P2[i,1], P2[j,1]]
    ax2.plot(x_coords, y_coords, 'k-', alpha=0.6, linewidth=1.5)

for i, label in enumerate(vertex_labels):
    ax2.annotate(label, (P2[i,0], P2[i,1]), xytext=(5, 5), 
                textcoords='offset points', fontsize=10, fontweight='bold')

ax2.set_title('Hexagonal Cayley Graph\nEdges = Adjacent transpositions', fontsize=12)
ax2.axis('equal')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./outputs/N3_permutohedron_hexagon.png', dpi=150, bbox_inches='tight')
plt.show()

# Validate geometric properties
print("N=3 Permutohedron Geometric Validation:")
print("=======================================")
print(f"Vertices (permutations): {len(P2)}")
print(f"Dimension of embedding space: {V.shape[1]} (= N-1 = {N-1})")
print(f"Cayley graph edges: {G3.number_of_edges()}")
print(f"Expected edges for N=3: {3*2//2} = 3") # N*(N-1)/2 for complete Cayley graph
print(f"Graph is connected: {nx.is_connected(G3)}")
print(f"All vertices have degree {3-1} = {N-1}: {all(G3.degree(i) == N-1 for i in G3.nodes())}")

# Verify the hexagon structure
angles = []
center = P2.mean(axis=0)
for point in P2:
    vec = point - center
    angle = np.arctan2(vec[1], vec[0])
    angles.append(angle)

angles = np.sort(angles)
angle_diffs = np.diff(angles)
angle_diffs = np.append(angle_diffs, 2*np.pi + angles[0] - angles[-1])

print(f"Vertex angles (degrees): {angles * 180/np.pi}")
print(f"Angular spacing: {angle_diffs * 180/np.pi} (should be ~60° each)")
print(f"Regular hexagon check: {np.allclose(angle_diffs, np.pi/3, atol=0.1)}")

print(f"\n✓ Permutohedron visualization saved to ./outputs/N3_permutohedron_hexagon.png")
print(f"✓ Geometric structure validates A₂ Coxeter system with rank N-1 = 2")

## 4. Takeaways for the N−1 derivation
- The state space of total orders lives on the **permutohedron** in the **sum-zero space** $V\cong\mathbb{R}^{N-1}$.
- Its **edges are adjacent reflections** (Coxeter generators), so the **rank** (= number of independent generators) is **$N-1$**.
- For $N=3$, this gives a clean, finitely checkable case: $A_2$ roots, hexagon permutohedron, and a 2-regular adjacent Cayley graph.

In **Notebook 2 (Geometry Derivation)** we crank this to $N=4$ to show the 3D permutohedron $\Pi_3$ and the full $A_3$ Cartan geometry — the first physics-relevant case (3D space).